## Data Transformation for Gold Lakehouse

#### Creating the_Customer Gold dimension tables

In [1]:
%%sql

CREATE OR REPLACE TEMPORARY VIEW tmpCustomers
AS
SELECT   CustomerID  CustomerKey
         ,FirstName
         ,LastName
         ,FullName
         ,DateInserted
FROM Sales_Silver.Customers

StatementMeta(, 94416d2a-13db-4b41-8ee1-76fe9f78baa2, 2, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [2]:
df = spark.sql("SELECT * FROM tmpCustomers")

# Saving the customer data to the Delta Lake path for DimCustomer in Sales_Gold Lakehouse
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/DimCustomer")

# Display the first 10 rows of the dataframe to preview your data
display(df.head(5))

StatementMeta(, 94416d2a-13db-4b41-8ee1-76fe9f78baa2, 4, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 06df2b03-0dd6-4898-bc9d-650d3c5a3155)

#### Creating the_Employee Gold dimension tables

In [2]:
%%sql

CREATE OR REPLACE TEMPORARY VIEW tmpEmployee
AS
SELECT   EmployeeID EmployeeKey
            ,ManagerID
            ,FirstName
            ,LastName
            ,FullName
            ,JobTitle
            ,OrganizationLevel
            ,MaritalStatus
            ,Gender
            ,Territory
            ,Country
            ,Group
FROM Sales_Silver.Employees

StatementMeta(, c40e0abf-f1fa-45fd-8d2c-898e272ada82, 4, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [3]:
df = spark.sql("SELECT * FROM tmpEmployee")
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/DimEmployee")

# Display the first 10 rows of the dataframe to preview your data
display(df.head(5))

StatementMeta(, c40e0abf-f1fa-45fd-8d2c-898e272ada82, 5, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 45c6a4fc-30c3-48cf-bc1f-07e5b7906aeb)

#### Creating the Products_Gold dimension tables

In [4]:
%%sql
--nb_DimProduct_Silver_Gold
CREATE OR REPLACE TEMPORARY VIEW tmpProducts
AS
SELECT    ProductID as ProductKey
            ,ProductNumber
            ,ProductName
            ,ModelName
            ,MakeFlag
            ,StandardCost
            ,ListPrice
            ,SubCategoryID
            ,DateInserted
FROM Sales_Silver.Products    

StatementMeta(, c40e0abf-f1fa-45fd-8d2c-898e272ada82, 6, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [5]:
df = spark.sql("SELECT * FROM tmpProducts")

df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/DimProduct")

# Display the first 5 rows of the dataframe to preview your data
display(df.head(5))

StatementMeta(, c40e0abf-f1fa-45fd-8d2c-898e272ada82, 7, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, b4ded402-dab9-43fc-977e-1ba8dc7dee5d)

#### Creating the ProductsCategories_Gold dimension tables

In [7]:
%%sql

CREATE OR REPLACE TEMPORARY VIEW tmpProductCategory
AS
SELECT       CategoryID as CategoryKey
            ,CategoryName
            ,DateInserted
FROM Sales_Silver.ProductCategories 

StatementMeta(, c40e0abf-f1fa-45fd-8d2c-898e272ada82, 9, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [8]:
df = spark.sql("SELECT * FROM tmpProductCategory")
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/DimProductCategory")

# Display the first 5 rows of the dataframe to preview your data
display(df.head(5))

StatementMeta(, c40e0abf-f1fa-45fd-8d2c-898e272ada82, 10, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 49dd7d29-f543-4216-a9a6-a2c14e046e8f)

#### Creating the ProductsSubCategories_Gold dimension tables

In [9]:
%%sql
--nb_DimProductSubCategories_Silver_Gold
CREATE OR REPLACE TEMPORARY VIEW tmpProductSubCategory
AS
SELECT      SubCategoryID as SubCategoryKey
            ,CategoryID as CategoryKey
            ,SubCategoryName
FROM Sales_Silver.ProductSubCategories

StatementMeta(, c40e0abf-f1fa-45fd-8d2c-898e272ada82, 11, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [10]:
df = spark.sql("SELECT * FROM tmpProductSubCategory")
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/DimProductSubCategory")

# Display the first 5 rows of the dataframe to preview your data
display(df.head(5))

StatementMeta(, c40e0abf-f1fa-45fd-8d2c-898e272ada82, 12, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 0c67a7c9-f2f5-4376-8679-153a61e42170)

#### Creating the FactSales_Gold dimension tables

In [11]:
%%sql
--nb_FactSales_Silver_Gold
CREATE OR REPLACE TEMPORARY VIEW tmpSales
AS
SELECT      SalesOrderID AS SalesOrderKey
            ,ifnull(e.EmployeeKey,-1) as EmployeeKey
            ,ifnull(c.CustomerKey,-1) as CustomerKey
            ,ifnull(p.ProductKey,-1) as ProductKey
            ,ifnull(pc.CategoryKey,-1) as CategoryKey
            ,ifnull(psc.SubCategoryKey,-1) as SubCategoryKey                       
            ,SalesOrderDetailID
            ,OrderDate
            ,DueDate
            ,ShipDate
            ,SubTotal
            ,TaxAmt
            ,Freight
            ,TotalDue
            ,OrderQty
            ,UnitPrice
            ,UnitPriceDiscount
            ,LineTotal
            ,o.DateInserted
FROM Sales_Silver.Orders o
LEFT JOIN Sales_Gold.DimCustomer c
ON c.CustomerKey =o.CustomerID

LEFT JOIN Sales_Gold.DimEmployee e 
ON e.EmployeeKey= o.EmployeeID

LEFT JOIN Sales_Gold.DimProduct p 
ON p.ProductKey = o.ProductID

LEFT JOIN Sales_Gold.DimProductSubCategory psc
ON psc.SubCategoryKey=p.SubCategoryID

LEFT JOIN Sales_Gold.DimProductCategory pc
ON pc.CategoryKey= psc.CategoryKey

StatementMeta(, c40e0abf-f1fa-45fd-8d2c-898e272ada82, 13, Finished, Available, Finished)

<Spark SQL result set with 0 rows and 0 fields>

In [12]:
df = spark.sql("SELECT * FROM tmpSales")
df.write.mode("overwrite").format("delta").option("overwriteSchema", "true").save("Tables/FactSales")

# Display the first 10 rows of the dataframe to preview your data
display(df.head(10))

StatementMeta(, c40e0abf-f1fa-45fd-8d2c-898e272ada82, 14, Finished, Available, Finished)

SynapseWidget(Synapse.DataFrame, 92267772-d660-47b6-b82a-182112c5d869)